**本文将从零开始实现一个RNN，并应用这个RNN模型来训练一个语言模型。**

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


`jaychou_lyrics.txt`文件中包含了周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》中的所有歌词，我们将使用这个数据集来训练RNN语言模型。

In [17]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    import zipfile
    with zipfile.ZipFile('./datasets/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
            
    ### 数据预处理
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:30000]       ## 取前30000个字作为训练数据。
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()   
print(list(char_to_idx.items())[:10])

[('释', 0), ('亮', 1), ('尸', 2), ('究', 3), ('常', 4), ('酱', 5), ('珍', 6), ('久', 7), ('样', 8), ('树', 9)]


In [20]:
def one_hot(x, n_class, dtype=torch.float32):
    x = x.long()
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device = x.device)
    
    result.scatter_(1, x.view(-1, 1), 1)
    return result

x = torch.tensor([0,2])
one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [22]:
# 将时间步拆开，将每个时间步中的batch_size大小的列向量进行one-hot编码。
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1783])


In [29]:
###初始化模型
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


will use cuda


In [30]:
####返回初始化的隐藏状态
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros( (batch_size, num_hiddens), device = device ), )


In [31]:
###定义模型
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H,W_hh) + b_h)
        Y = torch.matmul(H,W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [32]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape) 


5 torch.Size([2, 2582]) torch.Size([2, 256])


In [33]:
### 预测函数
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    
    ###初始化隐藏状态
    state = init_rnn_state(1, num_hiddens, device)
    
    output = [char_to_idx[prefix[0]]]
    
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [34]:
###测试
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)


'分开嘴刮裡腔燃泰3番傀村'

In [35]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)


In [41]:
###随机采样
def data_iter_random(corpus_indexs, batch_size, steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = len(corpus_indexs) - steps - 1
    epoch_size = num_examples // batch_size
    example_indexs = list(range(num_examples))
    random.shuffle(example_indexs)
    
    ## # 返回从pos开始的长为steps的序列
    def _data(pos):
        return corpus_indexs[pos: pos + steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    
    for i in range(epoch_size):
        ## 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indexs = example_indexs[i: i + batch_size]
        X = [_data(j) for j in batch_indexs]
        Y = [_data(j + 1) for j in batch_indexs]
        
        yield torch.tensor(X, dtype = torch.float32, device = device),\
                torch.tensor(Y, dtype = torch.float32, device =device) 
        
        
# 相邻采样
def data_iter_consecutive(corpus_indexs, batch_size, steps, device=None):  
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    corpus_indexs = torch.tensor(corpus_indexs, dtype=torch.float32, device=device)
    
    data_len = len(corpus_indexs) ## 29
    batch_len = data_len // batch_size  ##if batch_size = 2
    indexs = corpus_indexs[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // steps
    for i in range(epoch_size):
        i = i * steps
        X = indexs[:, i: i + steps]
        Y = indexs[:, i + 1: i + steps + 1]
        yield X, Y
        
## 定义优化函数
def sgd(params, lr, batch_size):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data


In [30]:
a = torch.ones((1,2)).view(-1)
a.size()

torch.Size([2])

In [23]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                           pred_len, prefixes):
    ## 判断选择哪种采样方式
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
        
    params = get_params()
    loss = nn.CrossEntropyLoss()

    ## 开始训练
    for epoch in range(num_epochs):
        state = (torch.zeros( (batch_size, num_hiddens), device = device ), ) ## 初始化隐藏状态
        
        ls_sum, n, start = 0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = (torch.zeros( (batch_size, num_hiddens), device = device ), )
            else:  
                # 使用detach_函数从计算图分离隐藏状态, 防止梯度计算开销太大
                for s in state:
                    s.detach_()

            inputs = to_onehot(X, vocab_size)  ## 将X转化为onehot的形式
            
            # outputs.size(): (num_steps, batch_size, vocan_size)
            # state.size(): (batch_size, num_hiddens) 
            (outputs, state) = rnn(inputs, state, params)
            
            # outputs.size(): (num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            
            # Y.size(): (batch_size, num_steps)
            # y.size(): (num_steps * batch_size) 1-dimention tensor.
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            
            # 使用交叉熵损失计算平均分类误差
            ls = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            ls.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            sgd(params, lr, 1)  
            ls_sum += ls.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(ls_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))


In [43]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']


In [44]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)


epoch 50, perplexity 38.506133, time 1.91 sec
 - 分开了天空 不用麻烦了 不用麻烦了 不用麻烦 用世界的希望 我们你的眼手 我用你的爱写 我用你的美写 我
 - 不分开 我只以放 也在我 我却指 我没有一种称　 是什么的小道 在你的那里 在风村 你说的 娘我最 我不了
epoch 100, perplexity 17.616453, time 2.08 sec
 - 分开始著 在我们乘半堡的天空 一点又重 我的感界 被不掉 让我用不来  你  功腐 豆夫  豆腐 豆夫 
 - 不分开 把爱的大味 我用无悔  永待回                                  
epoch 150, perplexity 11.593994, time 1.92 sec
 - 分开始天 说我的眼泪 我也想够不微笑 爱为我不知再想 你说你会别不到 我的多溢　 象你的Bl 远圣中期着
 - 不分开 微笑的大象 你的声音在我们一个人 你知定风 都是你的好量 你说你的手尖在西元前 深埋在美索不达米亚
epoch 200, perplexity 8.462342, time 1.84 sec
 - 分开始天 说伊的每牌 将我的空  回 的灵魂 你叫一起嗓   其他全部是垃圾 那多尾 你怎么 我说你豆回
 - 不分开 心来的大我全 雨呆的灵旧被 我在等你 也不需听  还的回忆 我手遇不见  不能就好自己 再象眼泪 
epoch 250, perplexity 6.435160, time 1.90 sec
 - 分开始天 手伊的正分　 败坏跟风 蝠不会听 出一秒 不需要要我 这里一口太后 一点 外的速板 我们 不懂
 - 不分开 戴爱的大象 你在颊微是 我却在你不见 不用麻耳   用h 一直 我翻速一直称　 我也悄出情 将炮里
